In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [2]:
#sqlite 設定
con = sqlite3.connect('../../data/race_data_set.db') # データベースに接続する
con.enable_load_extension(True)
con.load_extension("../../../../sqlite-amalgamation-3260000/libsqlitefunctions.so")
cur = con.cursor()

sql_result = "select race_id,stdev(last3f) as std3f,avg(last3f) as ave3f from race_result group by race_id"
sql_info   = "select * from race_info"
sql_pay = "select * from race_pay"

df_r = pd.read_sql_query(sql=sql_result,con=con)
df_i = pd.read_sql_query(sql=sql_info,con=con)

In [3]:
df = pd.read_csv("../../data/horse_data_set.csv")
df["pred_race_date"] = pd.to_datetime(df["pred_race_date"],format='%Y-%m-%d')
df["past_race_date"] = pd.to_datetime(df["past_race_date"],format='%Y-%m-%d')
df = df.merge(df_r,right_on="race_id",left_on="past_race_id",how="left")

In [4]:
#last3f偏差値計算
df["h_last3f"] = (((df["past_last3f"]-df["ave3f"])*10)/df["std3f"])*-1+50
del_col = ['race_id','ave3f', 'std3f']
df = df.drop(columns=del_col)

In [5]:
#性別のカテゴライズ
cate_sex  = {'牝':0,'牡':1,'セ':2}
df['pred_horse_sex'] = df['pred_horse_sex'].apply(lambda x : cate_sex.get(x))
df['past_horse_sex'] = df['past_horse_sex'].apply(lambda x : cate_sex.get(x))

In [6]:
#競馬場のカテゴライズ
cate_place = {'東京':0,'中山':1,'阪神':2,'京都':3,'小倉':4,'新潟':5,'福島':6,'中京':7,'札幌':8,'函館':9}
df['pred_place'] = df['pred_place'].apply(lambda x : cate_place.get(x))
df['past_place'] = df['past_place'].apply(lambda x : cate_place.get(x))

In [7]:
#馬場状態のカテゴライズ
cate_condition ={'良':0,'稍重':1,'稍':1,'重':2,'不良':3,'不':3}
df['pred_condition'] = df['pred_condition'].apply(lambda x : cate_condition.get(x))
df['past_condition'] = df['past_condition'].apply(lambda x : cate_condition.get(x))

In [8]:
#レースランク1のカテゴライズ
cate_rank1 = {'2歳':0,'3歳':1,'3歳以上':2,'4歳以上':3}
df['pred_race_rank1'] = df['pred_race_rank1'].apply(lambda x : cate_rank1.get(x))
df['past_race_rank1'] = df['past_race_rank1'].apply(lambda x : cate_rank1.get(x))

In [9]:
#レースランク2のカテゴライズ
cate_rank2   = {'新馬':0,'未勝利':1,'500万下':2,'1000万下':3,'1600万下':4,'オープン':5,'1勝クラス':2,'2勝クラス':3,'3勝クラス':4,}
df['past_race_rank2'] = df['past_race_rank2'].apply(lambda x : cate_rank2.get(x))
df['pred_race_rank2'] = df['pred_race_rank2'].apply(lambda x : cate_rank2.get(x))

In [10]:
#コースの種類のカテゴライズ
cate_kind = {'芝':0,'ダ':1}
df['pred_kind'] = df['pred_kind'].apply(lambda x : cate_kind.get(x))
df['past_kind'] = df['past_kind'].apply(lambda x : cate_kind.get(x))

In [11]:
#位置取りの計算
df['dif_past1'] = df["past_p4"] - df["past_tyaku"]
df['dif_past2'] = df["past_tyaku"] / df["past_p4"]

df['past_p1'] = df["past_p1"] / df["past_horse_count"]
df['past_p2'] = df["past_p2"] / df["past_horse_count"]
df['past_p3'] = df["past_p3"] / df["past_horse_count"]
df['past_p4'] = df["past_p4"] / df["past_horse_count"]
df['past_pg'] = df["past_tyaku"] / df["past_horse_count"]

In [12]:
#ローテーションの計算
df['lotation'] = (df["pred_race_date"] - df["past_race_date"]).dt.days

In [13]:
#ラップタイムの変換
df["lapf1"] = None
df["lapf2"] = None
df["lapf3"] = None
df["lapl2"] = None
df["lapl1"] = None
df[["lapf1","lapf2","lapf3","lapl2","lapl1"]] = [np.array(x.split('-'),dtype='float64')[[0,1,2,-2,-1]] for x in df.past_lap_time]
df["lapf1"] = [x[0]/200.0 if (x[1]/100)%2 == 0 else x[0]/100.0 for x in df[["lapf1","past_distance"]].values]
df[["lapf2","lapf3","lapl2","lapl1"]] = df[["lapf2","lapf3","lapl2","lapl1"]]/200.0

In [14]:
#距離延長、短縮
df["dif_distance"] = (df["pred_distance"] - df["past_distance"])/200

In [15]:
#距離分類
def distance_category(x):
    if x <= 1200:
        cate = 0
    elif x <= 1400:
        cate = 1
    elif x <= 1600:
        cate = 2
    elif x <= 1800:
        cate = 3
    elif x <= 2000:
        cate = 4
    elif x <= 2200:
        cate = 5
    elif x<=2400:
        cate = 6
    else:
        cate = 7
    return cate

df["pred_distance"] = df["pred_distance"].apply(lambda x : distance_category(x))
df["past_distance"] = df["past_distance"].apply(lambda x : distance_category(x))

In [17]:
merge_col=['pred_race_id', 'horse_id',
       'pred_tyaku', 'pred_umaban',
       'pred_horse_years', 'pred_horse_sex',
       'pred_race_rank1', 'pred_race_rank2', 'pred_kind', 'pred_distance',
       'pred_condition', 'pred_place', 'pred_dif_horse_weight']
df_past1 = df[(df["past_race_num"]==1)]
df_past2 = df[(df["past_race_num"]==2)]

#いらないカラムの削除
del_col = ['past_horse_weight','pred_horse_weight','past_race_id','horse_name','past_race_num','pred_wakuban','pred_race_date','past_tyaku','past_wakuban','past_race_date', 'past_horse_count', 'past_lap_time']
df_past1 = df_past1.drop(columns=del_col)
df_past2 = df_past2.drop(columns=del_col)
df_past = df_past1.merge(df_past2,on=merge_col,how='left')

In [107]:
#order_dataの作成
sql_result2 = "select race_id as pred_race_id,horse_id from race_result"
df_r = pd.read_sql_query(sql=sql_result2,con=con)

In [108]:
df_past1

,pred_race_id,horse_id,pred_tyaku,pred_umaban,pred_horse_years,pred_horse_sex,pred_race_rank1,pred_race_rank2,pred_kind,pred_distance,...,dif_past1,dif_past2,past_pg,lotation,lapf1,lapf2,lapf3,lapl2,lapl1,dif_distance
0,201601010103,2013105247,7,1,3,1,1,1,1,6,...,7,0.500000,0.437500,34,0.0635,0.0545,0.0655,0.0655,0.0670,3.0
2,201601010103,2013100742,6,2,3,1,1,1,1,6,...,-4,2.333333,0.466667,34,0.0720,0.0565,0.0645,0.0625,0.0650,1.5
10,201601010103,2013105711,11,3,3,1,1,1,1,6,...,2,0.714286,0.357143,41,0.0700,0.0565,0.0640,0.0640,0.0660,1.5
14,201601010103,2013100132,9,4,3,1,1,1,1,6,...,-2,1.181818,0.812500,97,0.0720,0.0590,0.0625,0.0660,0.0680,1.5
18,201601010103,2013101031,4,5,3,1,1,1,1,6,...,-1,1.125000,0.642857,14,0.0640,0.0580,0.0585,0.0590,0.0605,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012559,202010011212,2016106315,8,8,4,1,3,3,0,7,...,-2,3.000000,0.187500,7,0.0635,0.0610,0.0630,0.0615,0.0635,4.0
1012571,202010011212,2015104883,12,9,5,0,3,3,0,7,...,-1,1.083333,0.866667,105,0.0620,0.0595,0.0680,0.0605,0.0625,4.0
1012581,202010011212,2015104186,5,10,5,1,3,3,0,7,...,0,1.000000,0.307692,22,0.0650,0.0585,0.0595,0.0605,0.0640,0.0
1012588,202010011212,2014105441,3,11,6,1,3,3,0,7,...,5,0.444444,0.333333,42,0.0640,0.0600,0.0645,0.0585,0.0615,2.0


In [122]:
order = df_r.merge(df_past1,how='left',on=['pred_race_id','horse_id'])#.fillna(0)

In [123]:
order[order["pred_race_id"]==202010011212]

,pred_race_id,horse_id,pred_tyaku,pred_umaban,pred_horse_years,pred_horse_sex,pred_race_rank1,pred_race_rank2,pred_kind,pred_distance,...,dif_past1,dif_past2,past_pg,lotation,lapf1,lapf2,lapf3,lapl2,lapl1,dif_distance
8666,202010011212,2014101586,1.0,12.0,6.0,1.0,3.0,3.0,0.0,7.0,...,-4.0,5.000000,0.454545,64.0,0.0635,0.0545,0.0610,0.0615,0.0630,1.0
8667,202010011212,2016102005,2.0,2.0,4.0,0.0,3.0,3.0,0.0,7.0,...,1.0,0.500000,0.125000,21.0,0.0630,0.0595,0.0615,0.0625,0.0655,0.0
8668,202010011212,2014105441,3.0,11.0,6.0,1.0,3.0,3.0,0.0,7.0,...,5.0,0.444444,0.333333,42.0,0.0640,0.0600,0.0645,0.0585,0.0615,2.0
8669,202010011212,2014104329,4.0,4.0,6.0,1.0,3.0,3.0,0.0,7.0,...,-4.0,5.000000,0.416667,15.0,0.0630,0.0570,0.0610,0.0575,0.0595,3.0
8670,202010011212,2015104186,5.0,10.0,5.0,1.0,3.0,3.0,0.0,7.0,...,0.0,1.000000,0.307692,22.0,0.0650,0.0585,0.0595,0.0605,0.0640,0.0
8671,202010011212,2016105210,6.0,7.0,4.0,1.0,3.0,3.0,0.0,7.0,...,-3.0,2.000000,0.461538,22.0,0.0650,0.0585,0.0595,0.0605,0.0640,0.0
8672,202010011212,2015102790,7.0,6.0,5.0,1.0,3.0,3.0,0.0,7.0,...,3.0,0.727273,0.615385,22.0,0.0650,0.0585,0.0595,0.0605,0.0640,0.0
8673,202010011212,2016106315,8.0,8.0,4.0,1.0,3.0,3.0,0.0,7.0,...,-2.0,3.000000,0.187500,7.0,0.0635,0.0610,0.0630,0.0615,0.0635,4.0
8674,202010011212,2014106464,9.0,5.0,6.0,1.0,3.0,3.0,0.0,7.0,...,0.0,1.000000,0.153846,22.0,0.0650,0.0585,0.0595,0.0605,0.0640,0.0
8675,202010011212,2015104367,10.0,3.0,5.0,1.0,3.0,3.0,0.0,7.0,...,-1.0,1.250000,0.384615,22.0,0.0650,0.0585,0.0595,0.0605,0.0640,0.0


In [124]:
def rank_data_make(group_data):
    train_data = []
    label_data = []
    query_data = []
    c=0
    for name, group in group_data:
        data = group.sort_values(by="pred_umaban")
        train = data.values
        label = [i if i < 7 else 7 for i in data.pred_tyaku.values]
        query = len(data)
        train_data.append(train.tolist())
        label_data.append(label)
        query_data.append(query)
    return train_data,label_data,query_data

In [125]:
def train_split(train_data,test_data):
    train_data_s = []
    test_data_s = []
    for tr,te in zip(train_data,test_data):
        for tr,ter in zip(tr,te):
            train_data_s.append(tr)
            test_data_s.append(ter)
    return train_data_s,test_data_s

In [126]:
g,h,j=rank_data_make(order.groupby("pred_race_id"))

In [127]:
#train_data eval_data 分割
ind = np.arange(0,len(g))
train_index,test_index =  train_test_split(ind,shuffle=True)

In [128]:
train_r = np.array(g)
label_r = np.array(h)
query_r = np.array(j)
print(len(train_r),len(label_r),len(query_r))
train_rank,label_t_rank = train_split(train_r[train_index],label_r[train_index])
train_t_rank = np.array(train_rank)[:,3:-1]
label_t_rank = np.array(label_t_rank)
group_t_rank = query_r[train_index]

etra,ela = train_split(train_r[test_index],label_r[test_index])
train_e_rank = np.array(etra)[:,3:-1]
label_e_rank = np.array(ela) 
group_e_rank=query_r[test_index]

11415 11415 11415


In [135]:
params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'lambdarank',
    'metric': 'ndcg',   # for lambdarank
    'ndcg_eval_at': [1,2],  # for lambdarank
    'learning_rate': 0.01,
    'num_iterations':300,
    'verbosity': -1,
    'early_stopping_rounds': 10,
}

In [130]:
train_data = lgb.Dataset(train_t_rank, label = label_t_rank ,group=group_t_rank)
eval_data =  lgb.Dataset(train_e_rank ,label = label_e_rank ,group=group_e_rank, reference= train_data)

In [131]:
evals_result={}
evals_name=["train_cv_1","eval_cv_1"]
model      = lgb.train(params, train_data, valid_names=evals_name,valid_sets=[train_data,eval_data],evals_result=evals_result)

/home/h-kobayashi/anaconda3/envs/mypy/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/h-kobayashi/anaconda3/envs/mypy/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	train_cv_1's ndcg@1: 0.914969	train_cv_1's ndcg@2: 0.899835	train_cv_1's ndcg@3: 0.886783	eval_cv_1's ndcg@1: 0.92359	eval_cv_1's ndcg@2: 0.903334	eval_cv_1's ndcg@3: 0.891275
Training until validation scores don't improve for 10 rounds
[2]	train_cv_1's ndcg@1: 0.917817	train_cv_1's ndcg@2: 0.903307	train_cv_1's ndcg@3: 0.89069	eval_cv_1's ndcg@1: 0.926579	eval_cv_1's ndcg@2: 0.906874	eval_cv_1's ndcg@3: 0.895674
[3]	train_cv_1's ndcg@1: 0.9195	train_cv_1's ndcg@2: 0.903785	train_cv_1's ndcg@3: 0.891663	eval_cv_1's ndcg@1: 0.92473	eval_cv_1's ndcg@2: 0.907101	eval_cv_1's ndcg@3: 0.89471
[4]	train_cv_1's ndcg@1: 0.920044	train_cv_1's ndcg@2: 0.90407	train_cv_1's ndcg@3: 0.891636	eval_cv_1's ndcg@1: 0.925955	eval_cv_1's ndcg@2: 0.907577	eval_cv_1's ndcg@3: 0.896204
[5]	train_cv_1's ndcg@1: 0.923582	train_cv_1's ndcg@2: 0.905569	train_cv_1's ndcg@3: 0.892379	eval_cv_1's ndcg@1: 0.928703	eval_cv_1's ndcg@2: 0.908576	eval_cv_1's ndcg@3: 0.895521
[6]	train_cv_1's ndcg@1: 0.924087	train_c

In [134]:
#lambdarankのTEST
c=0
for name,group in order.groupby("pred_race_id"):
    c=c+1
    print("*"*20)
    test_data = group.iloc[:,3:-1]
    cd=(group.iloc[:,:3])
    cd["d"]=(model.predict(test_data.values))
    print(cd.sort_values(by='d', ascending=False))
    if c==20:
        break
    

********************
        pred_race_id    horse_id  pred_tyaku         d
137817  201601010103  2013104081        10.0  0.080750
137816  201601010103  2013100132         9.0  0.037960
137815  201601010103  2013104890         8.0  0.036917
137811  201601010103  2013101031         4.0  0.004380
137813  201601010103  2013100742         6.0 -0.029478
137808  201601010103  2013102336         1.0 -0.030090
137814  201601010103  2013105247         7.0 -0.047680
137812  201601010103  2013103498         5.0 -0.071612
137810  201601010103  2013103490         3.0 -0.080331
137818  201601010103  2013105711        11.0 -0.080331
137809  201601010103  2013103068         2.0 -0.106311
********************
        pred_race_id    horse_id  pred_tyaku         d
137826  201601010104  2013104154         8.0  0.052253
137829  201601010104  2013100870        11.0  0.019258
137819  201601010104  2013100206         1.0  0.010057
137821  201601010104  2013101518         3.0  0.004123
137823  201601010104  2

In [458]:
#データの読み込み
order = pd.read_csv("trainitit.csv")
#レース情報
sql_info   = "select * from race_info"
df_i = pd.read_sql_query(sql=sql_info,con=con)


In [512]:
#データをシャッフル
order=order.sample(frac=1)
#グループ分け
group_data= order.groupby("pred_race_id",sort=False)
#trainとqueryデータの作成
train_data = group_data.apply(lambda x : x.sort_values("pred_umaban"))#.reset_index(drop=True)
query_data = group_data.count()[["horse_id"]]#.reset_index()
#horse_countとqueryの数が一致しているデータのみ使用
q_d = query_data.reset_index().merge(df_i[["race_id","horse_count"]],left_on="pred_race_id",right_on="race_id",how="inner")
del_race_id = q_d[q_d["horse_id"]!=q_d["horse_count"]]["pred_race_id"].values

#データ不足のレースを削除
train_data=train_data.drop(index=del_race_id)#.reset_index(drop=True)
query_data=query_data.drop(index=del_race_id)
#train test split
id_set=train_data.pred_race_id.unique().tolist()
train_id ,test_id = train_test_split(id_set,train_size=0.8)
#train_data split
train = train_data.query('pred_race_id in {}'.format(train_id))
test  = train_data.query('pred_race_id in {}'.format(test_id))
#query split
q_train = query_data.query('pred_race_id in {}'.format(train_id))
q_test  = query_data.query('pred_race_id in {}'.format(test_id))

In [513]:
#lambdarank用にlightgbmデータを作成
x_train = train.iloc[:,2:-1].reset_index(drop=True).values
x_label = train.iloc[:,-1].reset_index(drop=True).values
x_query = q_train.horse_id.values
#test
y_train = test.iloc[:,2:-1].reset_index(drop=True).values
y_label = test.iloc[:,-1].reset_index(drop=True).values
y_query = q_test.horse_id.values

train_data = lgb.Dataset(x_train, label = x_label ,group=x_query)
eval_data =  lgb.Dataset(y_train ,label = y_label ,group=y_query, reference= train_data)

In [514]:
params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'lambdarank',
    'metric': 'ndcg',   # for lambdarank
    'ndcg_eval_at': [1,2],  # for lambdarank
    'learning_rate': 0.01,
    'num_iterations':3000,
    'verbosity': -1,
    'early_stopping_rounds': 1000,
}


array([[1.        , 0.1825    , 0.81818182, 0.54545455],
       [2.        , 0.1775    , 0.33333333, 0.5       ],
       [3.        , 0.179     , 0.36363636, 0.36363636],
       ...,
       [6.        , 0.1875    , 0.23076923, 0.23076923],
       [7.        , 0.1955    , 0.125     , 0.125     ],
       [8.        , 0.1805    , 0.75      , 0.33333333]])

In [517]:
evals_result={}
evals_name=["train_cv_1","eval_cv_1"]
model      = lgb.train(params, train_data, valid_names=evals_name,valid_sets=[train_data,eval_data],evals_result=evals_result)

/home/h-kobayashi/anaconda3/envs/mypy/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/h-kobayashi/anaconda3/envs/mypy/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	train_cv_1's ndcg@1: 0.342246	train_cv_1's ndcg@2: 0.394502	eval_cv_1's ndcg@1: 0.344703	eval_cv_1's ndcg@2: 0.393069
Training until validation scores don't improve for 1000 rounds
[2]	train_cv_1's ndcg@1: 0.36314	train_cv_1's ndcg@2: 0.412117	eval_cv_1's ndcg@1: 0.361326	eval_cv_1's ndcg@2: 0.403273
[3]	train_cv_1's ndcg@1: 0.372305	train_cv_1's ndcg@2: 0.418789	eval_cv_1's ndcg@1: 0.367927	eval_cv_1's ndcg@2: 0.405057
[4]	train_cv_1's ndcg@1: 0.37555	train_cv_1's ndcg@2: 0.419938	eval_cv_1's ndcg@1: 0.36297	eval_cv_1's ndcg@2: 0.405623
[5]	train_cv_1's ndcg@1: 0.377911	train_cv_1's ndcg@2: 0.42116	eval_cv_1's ndcg@1: 0.366379	eval_cv_1's ndcg@2: 0.404284
[6]	train_cv_1's ndcg@1: 0.3772	train_cv_1's ndcg@2: 0.420574	eval_cv_1's ndcg@1: 0.363922	eval_cv_1's ndcg@2: 0.403219
[7]	train_cv_1's ndcg@1: 0.377436	train_cv_1's ndcg@2: 0.420574	eval_cv_1's ndcg@1: 0.365994	eval_cv_1's ndcg@2: 0.405185
[8]	train_cv_1's ndcg@1: 0.379053	train_cv_1's ndcg@2: 0.421349	eval_cv_1's ndcg@1: 0.365

[101]	train_cv_1's ndcg@1: 0.410777	train_cv_1's ndcg@2: 0.438174	eval_cv_1's ndcg@1: 0.368426	eval_cv_1's ndcg@2: 0.407589
[102]	train_cv_1's ndcg@1: 0.41067	train_cv_1's ndcg@2: 0.438206	eval_cv_1's ndcg@1: 0.368326	eval_cv_1's ndcg@2: 0.407534
[103]	train_cv_1's ndcg@1: 0.411473	train_cv_1's ndcg@2: 0.438327	eval_cv_1's ndcg@1: 0.368787	eval_cv_1's ndcg@2: 0.408268
[104]	train_cv_1's ndcg@1: 0.411898	train_cv_1's ndcg@2: 0.438524	eval_cv_1's ndcg@1: 0.367897	eval_cv_1's ndcg@2: 0.408592
[105]	train_cv_1's ndcg@1: 0.411714	train_cv_1's ndcg@2: 0.438301	eval_cv_1's ndcg@1: 0.367545	eval_cv_1's ndcg@2: 0.408594
[106]	train_cv_1's ndcg@1: 0.411154	train_cv_1's ndcg@2: 0.438589	eval_cv_1's ndcg@1: 0.366147	eval_cv_1's ndcg@2: 0.408218
[107]	train_cv_1's ndcg@1: 0.412099	train_cv_1's ndcg@2: 0.439413	eval_cv_1's ndcg@1: 0.364625	eval_cv_1's ndcg@2: 0.407057
[108]	train_cv_1's ndcg@1: 0.412462	train_cv_1's ndcg@2: 0.43948	eval_cv_1's ndcg@1: 0.364648	eval_cv_1's ndcg@2: 0.406943
[109]	trai

[190]	train_cv_1's ndcg@1: 0.428632	train_cv_1's ndcg@2: 0.450122	eval_cv_1's ndcg@1: 0.371326	eval_cv_1's ndcg@2: 0.411122
[191]	train_cv_1's ndcg@1: 0.429012	train_cv_1's ndcg@2: 0.450201	eval_cv_1's ndcg@1: 0.372376	eval_cv_1's ndcg@2: 0.412244
[192]	train_cv_1's ndcg@1: 0.429343	train_cv_1's ndcg@2: 0.450241	eval_cv_1's ndcg@1: 0.372761	eval_cv_1's ndcg@2: 0.411537
[193]	train_cv_1's ndcg@1: 0.429924	train_cv_1's ndcg@2: 0.450455	eval_cv_1's ndcg@1: 0.372725	eval_cv_1's ndcg@2: 0.411121
[194]	train_cv_1's ndcg@1: 0.43054	train_cv_1's ndcg@2: 0.450754	eval_cv_1's ndcg@1: 0.373666	eval_cv_1's ndcg@2: 0.412037
[195]	train_cv_1's ndcg@1: 0.430667	train_cv_1's ndcg@2: 0.450966	eval_cv_1's ndcg@1: 0.373738	eval_cv_1's ndcg@2: 0.411918
[196]	train_cv_1's ndcg@1: 0.430344	train_cv_1's ndcg@2: 0.450761	eval_cv_1's ndcg@1: 0.372713	eval_cv_1's ndcg@2: 0.41159
[197]	train_cv_1's ndcg@1: 0.430693	train_cv_1's ndcg@2: 0.451141	eval_cv_1's ndcg@1: 0.371578	eval_cv_1's ndcg@2: 0.41168
[198]	train

[296]	train_cv_1's ndcg@1: 0.441385	train_cv_1's ndcg@2: 0.459059	eval_cv_1's ndcg@1: 0.36951	eval_cv_1's ndcg@2: 0.41373
[297]	train_cv_1's ndcg@1: 0.441233	train_cv_1's ndcg@2: 0.459134	eval_cv_1's ndcg@1: 0.369462	eval_cv_1's ndcg@2: 0.413443
[298]	train_cv_1's ndcg@1: 0.44141	train_cv_1's ndcg@2: 0.459352	eval_cv_1's ndcg@1: 0.370258	eval_cv_1's ndcg@2: 0.413638
[299]	train_cv_1's ndcg@1: 0.441913	train_cv_1's ndcg@2: 0.459289	eval_cv_1's ndcg@1: 0.370126	eval_cv_1's ndcg@2: 0.414013
[300]	train_cv_1's ndcg@1: 0.441738	train_cv_1's ndcg@2: 0.459911	eval_cv_1's ndcg@1: 0.370139	eval_cv_1's ndcg@2: 0.4147
[301]	train_cv_1's ndcg@1: 0.441832	train_cv_1's ndcg@2: 0.459816	eval_cv_1's ndcg@1: 0.370136	eval_cv_1's ndcg@2: 0.41536
[302]	train_cv_1's ndcg@1: 0.442382	train_cv_1's ndcg@2: 0.460127	eval_cv_1's ndcg@1: 0.370812	eval_cv_1's ndcg@2: 0.415475
[303]	train_cv_1's ndcg@1: 0.442555	train_cv_1's ndcg@2: 0.460242	eval_cv_1's ndcg@1: 0.369919	eval_cv_1's ndcg@2: 0.415235
[304]	train_cv

...

[707]	train_cv_1's ndcg@1: 0.483037	train_cv_1's ndcg@2: 0.492604	eval_cv_1's ndcg@1: 0.370805	eval_cv_1's ndcg@2: 0.409457
[708]	train_cv_1's ndcg@1: 0.483121	train_cv_1's ndcg@2: 0.492636	eval_cv_1's ndcg@1: 0.370129	eval_cv_1's ndcg@2: 0.409255
[709]	train_cv_1's ndcg@1: 0.483362	train_cv_1's ndcg@2: 0.49293	eval_cv_1's ndcg@1: 0.370033	eval_cv_1's ndcg@2: 0.409439
[710]	train_cv_1's ndcg@1: 0.483289	train_cv_1's ndcg@2: 0.493042	eval_cv_1's ndcg@1: 0.370033	eval_cv_1's ndcg@2: 0.409364
[711]	train_cv_1's ndcg@1: 0.483501	train_cv_1's ndcg@2: 0.49309	eval_cv_1's ndcg@1: 0.370612	eval_cv_1's ndcg@2: 0.409526
[712]	train_cv_1's ndcg@1: 0.483501	train_cv_1's ndcg@2: 0.493085	eval_cv_1's ndcg@1: 0.370033	eval_cv_1's ndcg@2: 0.409364
[713]	train_cv_1's ndcg@1: 0.483501	train_cv_1's ndcg@2: 0.493074	eval_cv_1's ndcg@1: 0.370033	eval_cv_1's ndcg@2: 0.409364
[714]	train_cv_1's ndcg@1: 0.483523	train_cv_1's ndcg@2: 0.492947	eval_cv_1's ndcg@1: 0.370033	eval_cv_1's ndcg@2: 0.409364
[715]	trai

[829]	train_cv_1's ndcg@1: 0.493517	train_cv_1's ndcg@2: 0.501047	eval_cv_1's ndcg@1: 0.3694	eval_cv_1's ndcg@2: 0.407647
[830]	train_cv_1's ndcg@1: 0.493697	train_cv_1's ndcg@2: 0.501016	eval_cv_1's ndcg@1: 0.3694	eval_cv_1's ndcg@2: 0.408325
[831]	train_cv_1's ndcg@1: 0.49389	train_cv_1's ndcg@2: 0.501142	eval_cv_1's ndcg@1: 0.3694	eval_cv_1's ndcg@2: 0.408223
[832]	train_cv_1's ndcg@1: 0.493872	train_cv_1's ndcg@2: 0.501129	eval_cv_1's ndcg@1: 0.370166	eval_cv_1's ndcg@2: 0.408172
[833]	train_cv_1's ndcg@1: 0.494228	train_cv_1's ndcg@2: 0.501313	eval_cv_1's ndcg@1: 0.369587	eval_cv_1's ndcg@2: 0.408134
[834]	train_cv_1's ndcg@1: 0.494229	train_cv_1's ndcg@2: 0.50131	eval_cv_1's ndcg@1: 0.369587	eval_cv_1's ndcg@2: 0.408451
[835]	train_cv_1's ndcg@1: 0.494175	train_cv_1's ndcg@2: 0.501513	eval_cv_1's ndcg@1: 0.369587	eval_cv_1's ndcg@2: 0.408408
[836]	train_cv_1's ndcg@1: 0.494332	train_cv_1's ndcg@2: 0.501587	eval_cv_1's ndcg@1: 0.369587	eval_cv_1's ndcg@2: 0.408431
[837]	train_cv_1

[931]	train_cv_1's ndcg@1: 0.500366	train_cv_1's ndcg@2: 0.506387	eval_cv_1's ndcg@1: 0.370158	eval_cv_1's ndcg@2: 0.409228
[932]	train_cv_1's ndcg@1: 0.500388	train_cv_1's ndcg@2: 0.506439	eval_cv_1's ndcg@1: 0.370158	eval_cv_1's ndcg@2: 0.409007
[933]	train_cv_1's ndcg@1: 0.500688	train_cv_1's ndcg@2: 0.506483	eval_cv_1's ndcg@1: 0.370158	eval_cv_1's ndcg@2: 0.408981
[934]	train_cv_1's ndcg@1: 0.500688	train_cv_1's ndcg@2: 0.506518	eval_cv_1's ndcg@1: 0.370061	eval_cv_1's ndcg@2: 0.408954
[935]	train_cv_1's ndcg@1: 0.500687	train_cv_1's ndcg@2: 0.506512	eval_cv_1's ndcg@1: 0.370061	eval_cv_1's ndcg@2: 0.408954
[936]	train_cv_1's ndcg@1: 0.500664	train_cv_1's ndcg@2: 0.506583	eval_cv_1's ndcg@1: 0.369959	eval_cv_1's ndcg@2: 0.409246
[937]	train_cv_1's ndcg@1: 0.500355	train_cv_1's ndcg@2: 0.506602	eval_cv_1's ndcg@1: 0.369959	eval_cv_1's ndcg@2: 0.409431
[938]	train_cv_1's ndcg@1: 0.500337	train_cv_1's ndcg@2: 0.506617	eval_cv_1's ndcg@1: 0.370055	eval_cv_1's ndcg@2: 0.409458
[939]	tr

[1055]	train_cv_1's ndcg@1: 0.505404	train_cv_1's ndcg@2: 0.510905	eval_cv_1's ndcg@1: 0.368243	eval_cv_1's ndcg@2: 0.407041
[1056]	train_cv_1's ndcg@1: 0.505548	train_cv_1's ndcg@2: 0.510918	eval_cv_1's ndcg@1: 0.368243	eval_cv_1's ndcg@2: 0.407041
[1057]	train_cv_1's ndcg@1: 0.5055	train_cv_1's ndcg@2: 0.511056	eval_cv_1's ndcg@1: 0.36822	eval_cv_1's ndcg@2: 0.407247
[1058]	train_cv_1's ndcg@1: 0.505956	train_cv_1's ndcg@2: 0.511034	eval_cv_1's ndcg@1: 0.369162	eval_cv_1's ndcg@2: 0.407246
[1059]	train_cv_1's ndcg@1: 0.505956	train_cv_1's ndcg@2: 0.511034	eval_cv_1's ndcg@1: 0.369162	eval_cv_1's ndcg@2: 0.407246
[1060]	train_cv_1's ndcg@1: 0.50598	train_cv_1's ndcg@2: 0.51106	eval_cv_1's ndcg@1: 0.369162	eval_cv_1's ndcg@2: 0.407253
[1061]	train_cv_1's ndcg@1: 0.50598	train_cv_1's ndcg@2: 0.511001	eval_cv_1's ndcg@1: 0.368391	eval_cv_1's ndcg@2: 0.406704
[1062]	train_cv_1's ndcg@1: 0.50581	train_cv_1's ndcg@2: 0.51112	eval_cv_1's ndcg@1: 0.368439	eval_cv_1's ndcg@2: 0.406746
[1063]	t

[1170]	train_cv_1's ndcg@1: 0.514229	train_cv_1's ndcg@2: 0.517157	eval_cv_1's ndcg@1: 0.369938	eval_cv_1's ndcg@2: 0.406133
[1171]	train_cv_1's ndcg@1: 0.514588	train_cv_1's ndcg@2: 0.517403	eval_cv_1's ndcg@1: 0.369938	eval_cv_1's ndcg@2: 0.406133
[1172]	train_cv_1's ndcg@1: 0.514635	train_cv_1's ndcg@2: 0.517728	eval_cv_1's ndcg@1: 0.369938	eval_cv_1's ndcg@2: 0.406133
[1173]	train_cv_1's ndcg@1: 0.514635	train_cv_1's ndcg@2: 0.517757	eval_cv_1's ndcg@1: 0.369938	eval_cv_1's ndcg@2: 0.406133
[1174]	train_cv_1's ndcg@1: 0.514635	train_cv_1's ndcg@2: 0.51773	eval_cv_1's ndcg@1: 0.369938	eval_cv_1's ndcg@2: 0.406133
[1175]	train_cv_1's ndcg@1: 0.514635	train_cv_1's ndcg@2: 0.517938	eval_cv_1's ndcg@1: 0.369938	eval_cv_1's ndcg@2: 0.406133
[1176]	train_cv_1's ndcg@1: 0.51467	train_cv_1's ndcg@2: 0.518016	eval_cv_1's ndcg@1: 0.369166	eval_cv_1's ndcg@2: 0.405691
[1177]	train_cv_1's ndcg@1: 0.514634	train_cv_1's ndcg@2: 0.518087	eval_cv_1's ndcg@1: 0.368418	eval_cv_1's ndcg@2: 0.405481
[1

In [518]:
#test_dataの実際のところを確認
def ppp(x):
    print(x.values)
    return model.predict([x.values])[0]
test.reset_index(drop=True).iloc[:,2:-1].apply(lambda x : ppp(x),axis=1)

[1.         0.1825     0.81818182 0.54545455]
[2.         0.1775     0.33333333 0.5       ]
[3.         0.179      0.36363636 0.36363636]
[4.         0.177      0.66666667 0.77777778]
[5.         0.177      0.88888889 0.88888889]
[6.    0.182 1.    1.   ]
[7.     0.1865 0.4375 0.25  ]
[8.         0.177      0.11111111 0.11111111]
[9.         0.1835     0.53846154 0.53846154]
[10.      0.1865  0.7     0.4   ]
[11.    0.19  0.9   1.  ]
[1.     0.188  0.1875 0.375 ]
[2.     0.174  0.9375 0.9375]
[3.         0.1705     0.38888889 0.33333333]
[4.     0.1775 0.1875 0.1875]
[5.         0.1685     0.55555556 0.55555556]
[6.         0.172      0.22222222 0.16666667]
[7.     0.1725 0.     0.    ]
[8.         0.169      0.55555556 0.38888889]
[9.         0.179      0.13333333 0.13333333]
[10.      0.1865  0.3125  0.75  ]
[11.          0.169       0.16666667  0.16666667]
[12.          0.172       0.94444444  0.94444444]
[13.          0.1685      0.11111111  0.11111111]
[14.          0.173       0.

[6.         0.1755     0.53333333 0.53333333]
[7.         0.1715     0.53333333 0.6       ]
[8.         0.177      0.5        0.57142857]
[9.         0.176      0.66666667 0.55555556]
[10.      0.1695  0.5625  0.6875]
[11.          0.177       0.71428571  0.71428571]
[12.      0.17    0.0625  0.0625]
[1.     0.1665 0.8125 0.5625]
[2.    0.164 0.75  0.625]
[3.    0.164 0.125 0.125]
[4.     0.1665 0.5    0.3125]
[5.         0.168      0.23529412 0.23529412]
[6.         0.1715     0.26666667 0.26666667]
[7.     0.1725 0.125  0.125 ]
[8.     0.1815 0.0625 0.0625]
[9.         0.1695     0.15384615 0.15384615]
[10.      0.1775  0.7     0.9   ]
[11.          0.1635      0.05555556  0.05555556]
[12.      0.174   0.875   0.8125]
[13.     0.174  0.125  0.125]
[14.          0.168       0.47058824  0.47058824]
[15.      0.1725  0.8125  0.75  ]
[16.     0.165  1.     1.   ]
[17.          0.1695      0.61538462  0.53846154]
[18.      0.164   0.25    0.3125]
[1.         0.185      0.28571429 0.285714

[1.     0.182  0.9375 0.9375]
[2.         0.1745     0.44444444 0.44444444]
[3.         0.1855     0.69230769 0.84615385]
[4.         0.173      0.33333333 0.4       ]
[5.         0.164      0.33333333 0.6       ]
[6.    0.171 0.875 0.75 ]
[7.     0.1695 1.     1.    ]
[8.     0.17   0.8125 0.8125]
[9.     0.168  0.8125 0.8125]
[10.          0.168       0.05555556  0.11111111]
[11.      0.174   0.5625  0.5625]
[12.          0.1725      0.73333333  0.73333333]
[13.      0.1885  0.1875  0.3125]
[14.      0.1715  0.8125  0.875 ]
[15.      0.184   0.8125  0.625 ]
[1.     0.1765 0.6875 0.625 ]
[2.         0.185      0.75       0.41666667]
[3.     0.175  0.3125 0.25  ]
[4.         0.1855     0.73333333 0.73333333]
[5.     0.186  0.6875 0.9375]
[6.    0.179 0.125 0.125]
[7.         0.187      1.         0.78571429]
[8.         0.195      0.21428571 0.85714286]
[9.         0.186      0.21428571 0.21428571]
[10.          0.185       1.          0.66666667]
[11.          0.18        0.53333333  

[11.     0.175  0.375  1.   ]
[12.          0.18        0.5         0.42857143]
[13.          0.1755      0.46666667  0.6       ]
[14.          0.1735      0.41666667  0.41666667]
[15.          0.178       0.73333333  0.73333333]
[16.          0.182       0.93333333  0.73333333]
[1.         0.1855     0.5        0.41666667]
[2.     0.183  0.4375 0.375 ]
[3.         0.1915     0.78571429 0.21428571]
[4.     0.183  0.5625 0.625 ]
[5.     0.187  0.9375 1.    ]
[6.         0.1855     0.84615385 0.76923077]
[7.         0.184      0.25       0.33333333]
[8.     0.1805 0.625  0.625 ]
[9.     0.182  0.6875 0.8125]
[10.      0.187   0.0625  0.0625]
[11.      0.1925  0.375   0.4375]
[12.          0.1755      0.66666667  0.66666667]
[13.      0.183   0.3125  0.1875]
[14.      0.1925  0.5     0.5625]
[1.     0.1795 0.5625 0.5   ]
[2.     0.1865 0.6    0.6   ]
[3.    0.179 0.625 0.875]
[4.     0.175  0.3125 0.3125]
[5.         0.1855     0.54545455 0.54545455]
[6.         0.1755     0.30769231 0.30

[12.          0.1705      0.69230769  0.76923077]
[13.          0.1635      0.16666667  0.16666667]
[14.          0.1635      0.83333333  0.94444444]
[15.          0.1635      0.61111111  0.55555556]
[16.          0.1635      0.16666667  0.16666667]
[17.          0.1705      0.81818182  0.90909091]
[1.     0.1745 0.875  0.875 ]
[2.    0.182 0.25  0.25 ]
[3.     0.1685 0.1875 0.1875]
[4.     0.174  0.3125 0.4375]
[5.     0.1745 0.125  0.125 ]
[6.     0.172  0.9375 1.    ]
[7.     0.1685 0.5625 0.625 ]
[8.         0.1795     0.44444444 0.5       ]
[9.     0.1685 0.4375 0.375 ]
[10.          0.182       0.13333333  0.26666667]
[11.      0.1745  0.2     0.2   ]
[12.      0.176   0.8125  0.75  ]
[13.      0.1745  0.3125  0.375 ]
[14.          0.1745      0.73333333  0.6       ]
[15.          0.1745      0.6         0.73333333]
[16.          0.1705      0.38888889  0.44444444]
[1.     0.193  0.75   0.6875]
[2.         0.1795     0.57142857 0.42857143]
[3.     0.1845 0.625  0.4375]
[4.     0.

...

[7.         0.186      0.73333333 0.6       ]
[8.    0.178 0.4   0.4  ]
[9.         0.171      0.61111111 0.61111111]
[10.      0.1795  0.375   0.1875]
[11.      0.1785  0.9375  0.9375]
[12.          0.187       0.13333333  0.26666667]
[13.          0.176       1.          0.90909091]
[14.          0.182       1.          0.93333333]
[15.          0.176       0.77777778  0.77777778]
[1.     0.19   0.0625 0.0625]
[2.     0.1785 0.6875 0.6875]
[3.     0.1785 0.5    0.5   ]
[4.     0.1785 0.5    0.5625]
[5.     0.1875 0.4375 0.625 ]
[6.     0.1785 0.9375 0.875 ]
[7.         0.1875     0.15384615 0.07692308]
[8.    0.183 0.75  0.625]
[9.     0.1795 0.125  0.125 ]
[10.          0.1795      0.33333333  0.53333333]
[11.          0.1715      0.64285714  0.64285714]
[12.      0.1695  0.3125  0.3125]
[13.      0.1755  0.1875  0.125 ]
[1.         0.164      0.53333333 0.6       ]
[2.         0.171      0.61111111 0.66666667]
[3.         0.1735     0.4        0.53333333]
[4.         0.169      0.7

[13.          0.1815      0.21428571  0.14285714]
[14.      0.185   0.25    0.6875]
[15.      0.1775  0.1875  0.1875]
[16.          0.1705      0.22222222  0.11111111]
[1.         0.177      0.84615385 0.76923077]
[2.    0.185 0.4   0.4  ]
[3.     0.1865 0.6    0.6   ]
[4.     0.1955 0.875  0.875 ]
[5.         0.181      0.5        0.42857143]
[6.         0.182      0.22222222 0.22222222]
[7.    0.183 0.125 0.125]
[8.         0.1795     0.66666667 0.4       ]
[9.     0.183  0.4375 0.1875]
[10.      0.1955  0.375   0.5   ]
[11.      0.1885  0.1875  0.1875]
[1.    0.183 1.    1.   ]
[2.    0.183 0.7   0.7  ]
[3.     0.1915 0.2    0.2   ]
[4.    0.183 0.9   0.9  ]
[5.         0.183      0.33333333 0.55555556]
[6.         0.1855     0.18181818 0.18181818]
[7.         0.178      0.72727273 0.63636364]
[8.         0.184      0.08333333 0.08333333]
[1.     0.18   0.125  0.3125]
[2.         0.183      0.54545455 0.18181818]
[3.         0.183      0.81818182 0.63636364]
[4.         0.1935     0

[7.     0.1675 0.8    0.8   ]
[8.     0.165  0.8125 0.6875]
[9.         0.167      0.88888889 0.88888889]
[10.          0.177       0.63636364  0.72727273]
[11.          0.169       0.33333333  0.33333333]
[12.      0.176   0.5     0.3125]
[13.      0.1785  0.625   0.4375]
[14.          0.167       0.66666667  0.66666667]
[15.      0.165   0.5     0.5625]
[16.          0.177       0.18181818  0.18181818]
[17.          0.17        0.05555556  0.05555556]
[18.          0.167       0.28571429  0.35714286]
[1.         0.1885     0.30769231 0.23076923]
[2.         0.173      0.64285714 0.14285714]
[3.     0.184  0.8125 0.4375]
[4.         0.19       1.         0.35294118]
[5.         0.188      0.07142857 0.35714286]
[6.   0.18 0.5  0.5 ]
[7.         0.178      0.27777778 0.27777778]
[8.         0.184      0.61538462 0.61538462]
[9.    0.173 0.    0.   ]
[10.      0.1815  0.75    0.5   ]
[11.          0.1965      0.63636364  0.81818182]
[12.      0.1735  0.625   0.5   ]
[13.          0.1835

[13.          0.187       0.46666667  0.33333333]
[14.          0.188       0.26666667  0.2       ]
[1.         0.183      0.84615385 0.76923077]
[2.         0.183      0.15384615 0.23076923]
[3.     0.186  0.0625 0.0625]
[4.    0.187 1.    0.875]
[5.         0.1925     1.         0.77777778]
[6.   0.19 0.25 0.25]
[7.         0.1865     0.35714286 0.07142857]
[1.     0.1895 0.125  0.125 ]
[2.         0.187      0.42857143 0.21428571]
[3.     0.1815 0.6875 0.5625]
[4.     0.1825 0.9375 1.    ]
[5.     0.1815 0.875  0.8125]
[6.     0.169  1.     0.9375]
[7.     0.186  0.0625 0.0625]
[8.     0.1815 0.3125 0.25  ]
[9.     0.1815 0.3125 0.375 ]
[10.      0.1815  0.9375  0.8125]
[11.      0.1845  0.1875  0.4375]
[12.     0.17   0.25   0.625]
[13.     0.182  0.4    0.4  ]
[14.          0.187       0.76923077  0.69230769]
[15.     0.17   0.125  0.125]
[1.     0.17   0.8125 0.8125]
[2.     0.175  0.1875 0.3125]
[3.     0.17   0.6875 0.6875]
[4.     0.1735 0.25   0.1875]
[5.         0.171      0

0        0.345768
1       -0.736454
2       -0.689880
3       -0.140212
4        0.365544
           ...   
18080   -1.098185
18081   -0.607102
18082   -0.975159
18083   -1.230528
18084    0.161725
Length: 18085, dtype: float64

In [528]:
test["pred"]=model.predict(([[1.0,0.1825,0.818182,0.545455]]))

ValueError: Length of values does not match length of index

In [529]:
test[test["pred_race_id"]==201808020210].sort_values(by="pred")

pred_race_id    horse_id  pred_umaban  race_first_3f  \
pred_race_id                                                                
201808020210 32584   201808020210  2010101884            8         0.1770   
             128383  201808020210  2010106549            2         0.1775   
             116706  201808020210  2014106132            7         0.1865   
             105789  201808020210  2012104833            3         0.1790   
             12837   201808020210  2014102607            9         0.1835   
             5609    201808020210  2014101095           10         0.1865   
             95451   201808020210  2013105471            4         0.1770   
             8038    201808020210  2013105930            1         0.1825   
             135747  201808020210  2012104772            5         0.1770   
             139463  201808020210  2013105900           11         0.1900   
             120590  201808020210  2012103942            6         0.1820   

                     first_past   past_p4  target      pred  
pred_race_id                                                 
201808020210 32584     0.111111  0.111111       2 -1.396151  
             128383    0.333333  0.500000       6 -0.736454  
             116706    0.437500  0.250000       8 -0.707222  
             105789    0.363636  0.363636       3 -0.689880  
             12837     0.538462  0.538462       9 -0.351301  
             5609      0.700000  0.400000       6 -0.143283  
             95451     0.666667  0.777778       5 -0.140212  
             8038      0.818182  0.545455       3  0.345768  
             135747    0.888889  0.888889       1  0.365544  
             139463    0.900000  1.000000       9  0.367844  
             120590    1.000000  1.000000      11  0.683843

In [190]:

label_t_rank = np.array(label_t_rank)
group_t_rank = query_r[train_index]

etra,ela = train_split(train_r[test_index],label_r[test_index])
train_e_rank = np.array(etra)[:,2:-1]
label_e_rank = np.array(ela) 
group_e_rank=query_r[test_index]

In [482]:
test.reset_index(drop=True).iloc[:,2:-1].apply(lambda x : print(type(model.predict([x.values])[0])),axis=1))

SyntaxError: invalid syntax (<ipython-input-482-0108b8cf9e58>, line 1)

In [417]:
q_d

,horse_id,race_id,horse_count
0,8,201606030105,9
1,16,201907040311,16
2,18,201707040412,18
3,15,201903010210,15
4,15,201604020702,15
...,...,...,...
10804,1,202010010305,9
10805,1,201606010506,16
10806,1,201908010603,11
10807,1,201607010210,16


In [173]:
train_data.[train_data["pred_race_id"] == [201601010103,202010011212]]

ValueError: Lengths must match to compare

In [230]:
train_data.sample(frac=1,replace=True)

,,pred_race_id,horse_id,pred_umaban,race_first_3f,first_past,past_p4,target
pred_race_id,,,,,,,,
201706010608,109809,201706010608,2011102118,3,0.1715,0.562500,0.687500,7
201607030212,121183,201607030212,2012106095,2,0.1740,0.307692,0.230769,9
201609050309,26954,201609050309,2013104544,8,0.1820,0.571429,0.714286,6
201705040412,59233,201705040412,2013100624,14,0.1735,0.500000,0.428571,11
201906040612,55704,201906040612,2016101865,13,0.1905,0.214286,0.142857,7
...,...,...,...,...,...,...,...,...
201804010408,67723,201804010408,2012101282,5,0.1705,0.066667,0.066667,1
201909020401,42354,201909020401,2016100525,11,0.1910,0.416667,0.250000,3
201807020212,62957,201807020212,2014106272,14,0.1750,0.266667,0.266667,4


In [268]:
train_data.index.get_level_values('pred_race_id')



Int64Index([201601010103, 201601010103, 201601010103, 201601010103,
            201601010103, 201601010103, 201601010103, 201601010103,
            201601010103, 201601010103,
            ...
            202010011212, 202010011212, 202010011212, 202010011212,
            202010011212, 202010011212, 202010011212, 202010011212,
            202010011212, 202010011212],
           dtype='int64', name='pred_race_id', length=139795)

In [269]:
train_data.index

MultiIndex([(201601010103, 107361),
            (201601010103,  48992),
            (201601010103,   2362),
            (201601010103,  46875),
            (201601010103,   4108),
            (201601010103,  28210),
            (201601010103,  51696),
            (201601010103,  82008),
            (201601010103, 123565),
            (201601010103,  49856),
            ...
            (202010011212,   4965),
            (202010011212,  11342),
            (202010011212,  15801),
            (202010011212,  28477),
            (202010011212,  80642),
            (202010011212,  66349),
            (202010011212, 126407),
            (202010011212, 125314),
            (202010011212,  42610),
            (202010011212,  12707)],
           names=['pred_race_id', None], length=139795)

In [281]:
g= order.groupby("pred_race_id",sort=False)
train_data = g.apply(lambda x : x.sort_values("pred_umaban"))

In [284]:
for i,j in g:
    print(j)
    break

        pred_race_id    horse_id  pred_umaban  race_first_3f  first_past  \
118002  202005020211  2017105893            2         0.1715    0.285714   
55442   202005020211  2017102192           15         0.1790    0.428571   
106208  202005020211  2017103666            6         0.1800    0.500000   
30947   202005020211  2017105426            7         0.1715    0.500000   
26380   202005020211  2017105619           12         0.1840    0.777778   
106719  202005020211  2017106193           17         0.1815    0.750000   
19127   202005020211  2017105212           16         0.1775    0.375000   
57753   202005020211  2017105186            9         0.1850    0.166667   
8026    202005020211  2017104784            8         0.1935    0.400000   
110816  202005020211  2017104478           10         0.1870    0.866667   
126926  202005020211  2017101431            3         0.1780    0.454545   
124943  202005020211  2017100029           14         0.1770    0.500000   
66030   2020

In [285]:
train_data.pred_race_id == 201610010406

pred_race_id        
202005020211  118002    False
              126926    False
              129290    False
              66030     False
              106208    False
                        ...  
201610010406  29698      True
202006010905  135511    False
201808010405  108985    False
201607010210  35358     False
201606010607  116446    False
Name: pred_race_id, Length: 139795, dtype: bool

In [287]:
query

,pred_race_id,horse_id
0,201601010103,11
1,201601010104,11
2,201601010106,7
3,201601010108,16
4,201601010109,13
...,...,...
10804,202010011208,15
10805,202010011209,10
10806,202010011210,14
10807,202010011211,14
